In [48]:
import matplotlib as plt
import numpy as np
import nltk
from nltk import CFG


NLP Assignment 3 - Nitzan Cohen & Gal Astrach

In [49]:

sg = """
S -> NP VP
DET -> 'the' | 'a' | 'A' | 'The' | 'Some'
VP -> IV | TV NP | GV NP NP
Noun ->   'boy' | 'boys' | 'book'
MNoun -> 'butter' | 'bread'
ADJ -> 'heavy'
PROPN -> 'John' | 'Mary' 
PRON -> 'They' | 'it' | 'she' | 'Everybody' | 'her' | 'them' | 'She'
NP -> MNoun |  DET Noun | DET ADJ Noun |  Noun | PRON  | PROPN 
IV -> 'left' 
GV -> 'gave'
TV -> 'eats' | 'loves' | 'love' |
"""
g = CFG.fromstring(sg)

# Bottom-up  parser
sr_parser = nltk.ShiftReduceParser(g, trace=2)

# Parse sentences and observe the behavior of the parser
def parse_sentence(sent):
    tokens = sent.split()
    trees = sr_parser.parse(tokens)
    for tree in trees:
        print(tree)

parse_sentence("John left")
parse_sentence("John eats bread")

Parsing 'John left'
    [ * John left]
  S [ 'John' * left]
  R [ PROPN * left]
  R [ NP * left]
  S [ NP 'left' * ]
  R [ NP IV * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (PROPN John)) (VP (IV left)))
Parsing 'John eats bread'
    [ * John eats bread]
  S [ 'John' * eats bread]
  R [ PROPN * eats bread]
  R [ NP * eats bread]
  S [ NP 'eats' * bread]
  R [ NP TV * bread]
  S [ NP TV 'bread' * ]
  R [ NP TV MNoun * ]
  R [ NP TV NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (PROPN John)) (VP (TV eats) (NP (MNoun bread))))


    John loves Mary
    They love Mary
    They love her
    She loves them
    Everybody loves John
    A boy loves Mary
	The boy loves Mary
	Some boys love Mary
    John gave Mary a heavy book
    John gave it to Mary
	John likes butter
	John moves a chair

In [50]:
parse_sentence('John loves Mary')
parse_sentence('They love Mary')
parse_sentence('They love her')
parse_sentence('She loves them')
parse_sentence('Everybody loves John')
parse_sentence('A boy loves Mary')
parse_sentence('The boy loves Mary')
parse_sentence('Some boys love Mary')
parse_sentence('John gave Mary a heavy book')
# parse_sentence('John gave it to Mary')
# parse_sentence('John likes butter')
# parse_sentence('John moves a chair')

Parsing 'John loves Mary'
    [ * John loves Mary]
  S [ 'John' * loves Mary]
  R [ PROPN * loves Mary]
  R [ NP * loves Mary]
  S [ NP 'loves' * Mary]
  R [ NP TV * Mary]
  S [ NP TV 'Mary' * ]
  R [ NP TV PROPN * ]
  R [ NP TV NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (PROPN John)) (VP (TV loves) (NP (PROPN Mary))))
Parsing 'They love Mary'
    [ * They love Mary]
  S [ 'They' * love Mary]
  R [ PRON * love Mary]
  R [ NP * love Mary]
  S [ NP 'love' * Mary]
  R [ NP TV * Mary]
  S [ NP TV 'Mary' * ]
  R [ NP TV PROPN * ]
  R [ NP TV NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (PRON They)) (VP (TV love) (NP (PROPN Mary))))
Parsing 'They love her'
    [ * They love her]
  S [ 'They' * love her]
  R [ PRON * love her]
  R [ NP * love her]
  S [ NP 'love' * her]
  R [ NP TV * her]
  S [ NP TV 'her' * ]
  R [ NP TV PRON * ]
  R [ NP TV NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (PRON They)) (VP (TV love) (NP (PRON her))))
Parsing 'She loves them'
    [ * She loves them]
  S [ 'She' * loves 

In [51]:
from nltk import PCFG
from nltk.grammar import Nonterminal
from collections import defaultdict
from nltk.probability import DictionaryProbDist
from nltk import Tree

def generate_Tree(symbol,total_dicts):
    if type(symbol) != Nonterminal:
        return symbol
    else:
        children = total_dicts[symbol].generate()
        return Tree(symbol,[generate_Tree(expander,total_dicts) for expander in children])

grammer = """
    S -> NP VP [1.0]
    NP -> Det N [0.5] | NP PP [0.25] | 'John' [0.1] | 'I' [0.15]
    Det -> 'the' [0.8] | 'my' [0.2]
    N -> 'man' [0.5] | 'telescope' [0.5]
    VP -> VP PP [0.1] | V NP [0.7] | V [0.2]
    V -> 'ate' [0.35] | 'saw' [0.65]
    PP -> P NP [1.0]
    P -> 'with' [0.61] | 'under' [0.39]
"""
def pcfg_generate(grammar):
    pcfg = PCFG.fromstring(grammar)
    productions = pcfg.productions()
    total_dicts = {}
    collect_productions = defaultdict(lambda : {})
    for production in productions:
        collect_productions[production.lhs()][production.rhs()] = production.prob()
    for lhs,rhs in collect_productions.items():
        total_dicts[lhs] = DictionaryProbDist(collect_productions[lhs])
    return generate_Tree(Nonterminal('S'),total_dicts)
    
#(S (NP I) (VP (V saw)))
for i in range(1,5):
    print("----------")
    print(pcfg_generate(grammer))
    print("----------")
print('---')



----------
(S (NP (Det the) (N man)) (VP (V saw) (NP (Det the) (N telescope))))
----------
----------
(S (NP (Det the) (N man)) (VP (V saw) (NP I)))
----------
----------
(S (NP (Det the) (N man)) (VP (V ate)))
----------
----------
(S (NP (Det the) (N telescope)) (VP (V ate) (NP I)))
----------
---
